In [1]:
from langchain.schema import Document
import pandas as pd
import numpy as np
from langchain.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [3]:
df = pd.read_csv('data/fulldata.csv')
df.head()

,category,query,solution
0,How to Book,How can I book a flight with Air India?,"Booking a flight with Air India is a breeze, a..."
1,How to Book,What are the steps to book an Air India flight...,"Booking an Air India flight is quite easy, whe..."
2,How to Book,Can I book a trip with multiple destinations?,"Absolutely, you can book a trip with multiple ..."
3,How to Book,Is there a time limit for booking tickets online?,You can take advantage of online booking for o...
4,Passenger Detail,How many passengers can I add in a single book...,"When booking a flight with us, you can add up ..."


In [4]:
df.drop_duplicates(inplace=True,subset=['query'])
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=['index'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  175 non-null    object
 1   query     175 non-null    object
 2   solution  175 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


In [5]:
df.head()

,category,query,solution
0,How to Book,How can I book a flight with Air India?,"Booking a flight with Air India is a breeze, a..."
1,How to Book,What are the steps to book an Air India flight...,"Booking an Air India flight is quite easy, whe..."
2,How to Book,Can I book a trip with multiple destinations?,"Absolutely, you can book a trip with multiple ..."
3,How to Book,Is there a time limit for booking tickets online?,You can take advantage of online booking for o...
4,Passenger Detail,How many passengers can I add in a single book...,"When booking a flight with us, you can add up ..."


In [6]:
doc_list = []
for i in range(len(df)):

    id = i
    q = df.iloc[i]['query']
    s = df.iloc[i]['solution']
    doc = Document(page_content=s,metadata={'id':i,'question':q})
    doc_list.append(doc)

In [7]:
doc_list[5]

Document(page_content='If you do not have a surname or last name, enter your single name in the ‘Last Name’ and ‘First Name’ fields.\n\nExample:\xa0If your name is\xa0John, type\xa0John\xa0in both fields, so it appears on your ticket as John/John.\n\nCountry-specific rules for single names\n\nThe US and Canada:\xa0If you have a single name (for example, John), please enter your ‘Last Name’ as\xa0John\xa0and ‘First Name’ as\xa0FNU\xa0to read as John/FNU.\n\nAustralia:\xa0If you have a single name (for example, Gayatri), please enter your ‘Last Name’ as\xa0Gayatri\xa0and ‘First Name’ as\xa0Unknown\xa0so it appears on your ticket as Gayatri/Unknown.\n\nThe UAE:\xa0If you have a single name (for example, Sunil), please enter your ‘First Name’ and ‘Last Name’ as\xa0Sunil\xa0so it appears on your ticket as Sunil/Sunil.', metadata={'id': 5, 'question': "How should I enter my name if I don't have a surname?"})

In [8]:
docstore = FAISS.from_documents(doc_list, embedding=embeddings)

In [9]:
docstore.save_local('solutions')